In [2]:
import pandas as pd
import mysql.connector
from dotenv import load_dotenv
import os

# Load env vars
load_dotenv(override=True)

username = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
host = "localhost"
port = 3306
database = "student_performance_db"

# Load Dataset
df = pd.read_csv("./datasets/student_performance.csv")

# Connect to MySQL
conn = mysql.connector.connect(
    host=host,
    user=username,
    password=password,
    database=database,
    port=port
)
cursor = conn.cursor()

# -----------------------------
# Lookup Tables
# -----------------------------

# Gender
cursor.execute("""
CREATE TABLE IF NOT EXISTS gender (
    id INT PRIMARY KEY,
    text VARCHAR(20)
)
""")
gender_values = [(0, 'Female'), (1, 'Male'), (2, 'Other')]
cursor.executemany("INSERT IGNORE INTO gender VALUES (%s, %s)", gender_values)

# Education Level
cursor.execute("""
CREATE TABLE IF NOT EXISTS education_level (
    id INT PRIMARY KEY,
    text VARCHAR(30)
)
""")
education_values = [(0, 'High School'), (1, 'Undergraduate'), (2, 'Postgraduate')]
cursor.executemany("INSERT IGNORE INTO education_level VALUES (%s, %s)", education_values)

# Course Name
cursor.execute("""
CREATE TABLE IF NOT EXISTS course_name (
    id INT PRIMARY KEY,
    text VARCHAR(50)
)
""")
course_values = [
    (0, 'Python Basics'),
    (1, 'Machine Learning'),
    (2, 'Data Science'),
    (3, 'Web Development'),
    (4, 'Cybersecurity')
]
cursor.executemany("INSERT IGNORE INTO course_name VALUES (%s, %s)", course_values)

# Engagement Level
cursor.execute("""
CREATE TABLE IF NOT EXISTS engagement_level (
    id INT PRIMARY KEY,
    text VARCHAR(20)
)
""")
engagement_values = [(0, 'Low'), (1, 'Medium'), (2, 'High')]
cursor.executemany("INSERT IGNORE INTO engagement_level VALUES (%s, %s)", engagement_values)

# Learning Style
cursor.execute("""
CREATE TABLE IF NOT EXISTS learning_style (
    id INT PRIMARY KEY,
    text VARCHAR(30)
)
""")
learning_values = [
    (0, 'Visual'),
    (1, 'Reading/Writing'),
    (2, 'Auditory'),
    (3, 'Kinesthetic')
]
cursor.executemany("INSERT IGNORE INTO learning_style VALUES (%s, %s)", learning_values)

# Dropout Likelihood
cursor.execute("""
CREATE TABLE IF NOT EXISTS dropout_likelihood (
    id INT PRIMARY KEY,
    text VARCHAR(10)
)
""")
dropout_values = [(0, 'No'), (1, 'Yes')]
cursor.executemany("INSERT IGNORE INTO dropout_likelihood VALUES (%s, %s)", dropout_values)

print("Lookup values inserted successfully.")

# -----------------------------
# Main Student Performance Table
# -----------------------------
cursor.execute("""
CREATE TABLE IF NOT EXISTS student_performance (
    Student_ID VARCHAR(10) PRIMARY KEY,
    Age INT,
    Gender INT,
    Education_Level INT,
    Course_Name INT,
    Time_Spent_on_Videos INT,
    Quiz_Attempts INT,
    Quiz_Scores INT,
    Forum_Participation INT,
    Assignment_Completion_Rate INT,
    Engagement_Level INT,
    Final_Exam_Score INT,
    Learning_Style INT,
    Feedback_Score INT,
    Dropout_Likelihood VARCHAR(3),
    FOREIGN KEY (Gender) REFERENCES gender(id),
    FOREIGN KEY (Education_Level) REFERENCES education_level(id),
    FOREIGN KEY (Course_Name) REFERENCES course_name(id),
    FOREIGN KEY (Engagement_Level) REFERENCES engagement_level(id),
    FOREIGN KEY (Learning_Style) REFERENCES learning_style(id)
)
""")

# -----------------------------
# Prepare Values for Insert
# -----------------------------
mapping_dicts = {
    "Gender": {"Female": 0, "Male": 1, "Other": 2},
    "Education_Level": {"High School": 0, "Undergraduate": 1, "Postgraduate": 2},
    "Course_Name": {
        "Python Basics": 0, "Machine Learning": 1, "Data Science": 2,
        "Web Development": 3, "Cybersecurity": 4
    },
    "Engagement_Level": {"Low": 0, "Medium": 1, "High": 2},
    "Learning_Style": {"Visual": 0, "Reading/Writing": 1, "Auditory": 2, "Kinesthetic": 3}
}

for col, mapping in mapping_dicts.items():
    df[col] = df[col].map(mapping)

# Convert DataFrame into tuples
values = [tuple(x) for x in df.to_numpy()]

# Insert into main table
sql_insert = """
INSERT INTO student_performance (
    Student_ID, Age, Gender, Education_Level, Course_Name,
    Time_Spent_on_Videos, Quiz_Attempts, Quiz_Scores, Forum_Participation,
    Assignment_Completion_Rate, Engagement_Level, Final_Exam_Score,
    Learning_Style, Feedback_Score, Dropout_Likelihood
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
cursor.executemany(sql_insert, values)
conn.commit()

print(f"{cursor.rowcount} student records inserted successfully.")

# Close connection
cursor.close()
conn.close()
print("Database connection closed.")


Lookup values inserted successfully.
10000 student records inserted successfully.
Database connection closed.
